# B. Etude approfondie du cas Auto-mpg

Effectuer le même genre d'opération sur le jeu de données UCI Auto MPG, consistant à prédire la consommation (MPG) d'un ensemble de voitures.

https://archive.ics.uci.edu/ml/datasets/auto+mpg

1. Tester un régresseur de base sur le sous-ensemble de colonnes correspondant à des données numériques 
2. Encoder les colonnes catégorielles et comparer la performance
3. Que penser des colonnes de date et de cylindrée?
    * Tester leur prise en compte numérique *vs* leur prise en compte catégorielle
4. La dernière colonne est séparable en plusieurs (marque, modèle)
    * Quel impact sur les performances
5. Imaginer d'autres manières de prendre en compte les valeurs manquantes



In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Chargement des données (1/2)

filename = "data/auto-mpg.csv"
data = pd.read_csv(filename)
X = data.values[:,1:]
Y = data.values[:,0]

# la colonne 2 contient des données manquantes (?)... Et elle est encodée en chaine de caractères...
data.loc[30:40, :] # equiv. dans pandas de: print(X[30:40])

# alternative : data.head(50)

## B.1. La gestion des données manquantes

De nombreux jeux de données présentent des données manquantes, souvent dans une ou deux colonnes et sur quelques lignes. La question de savoir comment traiter ce cas de figure occupe beaucoup la communauté.

1. Du point de vue théorique, ça relève de l'algorithme EM pour l'estimation de valeur manquante
2. Du point de vue très opérationnel, on peut:
    * supprimer les lignes affectées
    * affecter la valeur moyenne de la colonne aux cases manquantes (cf case ci-dessous)
3. D'autres stratégies assez simples sont disponibles dans sk-learn

**1er problème opérationnel:** la colonne contenant des données manquantes est transformée en chaine de caractères car certaines valeurs valent `'?'`


In [ ]:
# Chargement des données (2/2)

# On propose de les remplacer par la valeur moyenne de la colonne [A LA MAIN]
# 1. On remplace par des 0
X[:,2] = [0 if X[i,2] == '?' else int(X[i,2]) for i in range (len(X))]
# 2. On remplace par la moyenne
m = np.round(X[:,2].sum()/np.where(X[:,2]>0, 1, 0).sum()) # 104
X[:,2] = np.where(X[:,2]==0, m, X[:,2])

print(X[30:40])

In [ ]:
# Chargemnet des données (3/3)
# la méthode ci-dessus est fonctionnelle mais pas évidente à coder: il est possible d'obtenir la même chose avec une petite option de pandas

filename = "data/auto-mpg.csv"
data = pd.read_csv(filename, na_values = '?') # => transformer les valeurs manquantes en NaN
                                              # toutes les colonnes seront numériques
X = data.values[:,1:]
Y = data.values[:,0]

print(X[30:40]) # avec valeur manquante

Stratégies alternatives **à maitriser** :

https://scikit-learn.org/stable/modules/classes.html#module-sklearn.impute

Le réglage de la boite ci-dessous est à `mean`... N'hésitez pas à comparer avec d'autres réglages!

In [ ]:
# application directe de la fonction:

from sklearn.impute import SimpleImputer

imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean') # attention, fonction par robuste aux str
Xfill = imp_mean.fit_transform(X[:,2:3]) # traitemnet de la colonne 2
X[:,2:3] = Xfill

print(X[30:40])
# que se passe-t-il?

## B.2. Analyse du contenu 

Nous nous intéressons d'abord à la dernière colonne qui est problématique (car non numérique)... Puis à toutes les autres colonnes

In [ ]:
# analyse de la dernière colonne (chaine de caractères):
print("Combien de valeurs uniques:", len(np.unique(X[:,-1])), "(nb val=",len(X),")")

print(X[:5,-1]) # affichage de 5 valeurs

In [ ]:
#Il faut sans doute se limiter à la marque:
# remplacer le nom complet du modèle par la marque seule (cette information peut avoir de la valeur pour estimer la consommation)

###  TODO  ###

print(X[:5,-1]) # affichage de 5 valeurs (corrigées avec juste la marque)

### Visualisation des données + target encoding

L'outil premier pour prendre en main les données est l'histogramme: dans l'exemple ci-dessous, on trace les histogrammes de répartition des données (en bleu).

On ajoute une seconde information: la valeur moyenne de la cible pour chaque barre de l'histogramme (ça ressemble beaucoup à du *target encoding*). Idée: on essaie de voir si les valeurs de la cibles *bougent* par rapport aux catégories de l'histogramme.

In [ ]:

# Analyse des colonnes (histogramme)
plt.figure(figsize=[5*X.shape[1],5])
for i in range(X.shape[1]):
    plt.subplot(1,X.shape[1],i+1)
    # plt.figure()
    if i<X.shape[1]-1: # colonne numériques
        nbins = np.minimum(len(np.unique(X[:,i])), 12) # extraction des colonnes à valeurs discrètes
        [e,x] = np.histogram(X[:,i], nbins)
        x[-1] += 0.001
        conso = [Y[np.logical_and(X[:,i]>=x[j], X[:,i]<x[j+1])].mean() for j in range(len(e))]
        # print(e,x)
        plt.bar(np.arange(len(e)),e)
        plt.bar(np.arange(len(e)),conso, alpha=0.5)
        plt.title('Colonne '+str(i))
    else: # colonnes textuelles
        val,e = np.unique(X[:,-1], return_counts=True)
        plt.bar(np.arange(len(e)),e)
        conso = [Y[X[:,-1] == v].mean() for v in val]
        plt.bar(np.arange(len(e)),conso, alpha=0.5)
        plt.title('Colonne '+str(i))

#plt.savefig('fig/auto-mpg-all.png')
plt.show()


## B.3. Performances de base puis enrichissement des données

On calcule d'abord une performance sur les données numériques faciles à exploiter... Puis on va enrichir la représentation

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.preprocessing import OneHotEncoder


# Construction d'un modèle de référence sur les colonnes numériques
Xr = X[:,1:6]
mod = Ridge()

sc = cross_val_score( mod, Xr, Y, scoring='neg_mean_absolute_percentage_error')

print("modèle de référence : ",sc, sc.mean())
print(" dimension des données: ", Xr.shape)

Objectif de la boite ci-dessous: arriver dans les 12% d'erreur

In [ ]:
# Transfromation des données et Amélioration (?) des résultats

enc = OneHotEncoder(handle_unknown='ignore', sparse_output=False) # aller voir la document (ou ex plus haut)

for col  in [0,5,6,7]: # pour chacune de ces colonnes discrètes, 
    # Construire la matrice Xtmp one-hot des occurences de variables discrètes
    # Concaténer Xtmp avec Xr
    # Calculer les performances du modèle à chaque itération
    ###  TODO  ###



## B.4. Histogramme de valeurs

Il est parfois intéressant de convertir une variable continue en un histogramme de valeurs. Sur la colonne 1 des données, on avait l'impression qu'il y avait des comportements un peu similaire par groupe de valeurs... On a donc ajouté des catégories.

Objectif de la boite ci-dessous: arriver dans les 11% d'erreur

In [ ]:
# Transformation de la caractéristique 5 (Année)
from sklearn.preprocessing import KBinsDiscretizer

dim = 1 # dimension à encoder
#print(X[:10,dim])
enc = KBinsDiscretizer(n_bins=4, encode='onehot', strategy='uniform') # comprendre la signification des arguments
Xtmp = enc.fit_transform(X[:,dim].reshape(-1,1)).toarray()
#print(Xtmp[:10,:])
Xr = np.concatenate((Xr, Xtmp), axis = 1)
sc = cross_val_score( mod, Xr, Y, scoring='neg_mean_absolute_percentage_error')
print(Xtmp.shape)
print("modèle de référence : ",sc, sc.mean())


## B.5. Transformations arbitraires

Enrichir les données construisant une colonne binaire qui vaut 1 pour les voitures de 4 cylindres de moins de 2500 en masse

In [ ]:
# créer une colonne biaire qui recense les petites voitures (masse<2500) de 4 cylindres (=1 vs 0 partout ailleurs)

Xtmp = np.where(np.logical_and(X[:,0]==4, X[:,4]<2500), 1, 0).reshape(-1,1)
Xr = np.concatenate((Xr, Xtmp), axis = 1)
sc = cross_val_score( mod, Xr, Y, scoring='neg_mean_absolute_percentage_error')
print(Xtmp.shape)
print("modèle de référence : ",sc, sc.mean())


## B.6. Passage à un modèle de l'état de l'art

Est ce que les variables construites ont du sens pour un modèle non-linéaire de type XGBoost/catboost?


In [ ]:
# Passage à un modèle de l'état de l'art

# catboost

import catboost as cgb # !pip install xgboost # en cas de besoin

bst = cgb.CatBoostRegressor()
sc = cross_val_score( bst, Xr, Y, scoring='neg_mean_absolute_percentage_error')
print("catboost : ",sc, sc.mean())

sc = cross_val_score( bst, X[:,:6], Y, scoring='neg_mean_absolute_percentage_error')
print("catboost (X origine): ",sc, sc.mean())

In [ ]:
# !pip install catboost 

# Construction du sujet à partir de la correction

In [1]:
###  TODO )"," TODO ",\
    txt, flags=re.DOTALL))
f2.close()

### </CORRECTION> ###